In [1]:
\l E:\pycharm\PycharmProjects\kdbq\columbiaHdb\2019.01.03 / Loading and Checking data

In [2]:
/ Filling nan
bin10: update reverse fills reverse date,
     reverse fills reverse mid,
     reverse fills reverse spread,
     reverse fills reverse vol,
     reverse fills reverse adv
     by id from bin10
select  from bin10

time     id date       trade     mid      spread        vol         adv      ..
-----------------------------------------------------------------------------..
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e..
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e..
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e..
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e..
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e..
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e..
09:31:00 0  2019.01.03 -22823.01 91.56276 0.000663703   0.000446367 1.122415e..
09:31:10 0  2019.01.03 -4940.182 91.59088 0.0005615949  0.000446367 1.122415e..
09:31:20 0  2019.01.03 -41691.88 91.59322 0.0005360678  0.000446367 1.122415e..
09:31:30 0  2019.01.03 -539826.4 91.60727 0.0007402842  0.000446367 1.122415e..
09:31:40 0  2019.01.03 -53883.89 91.5487

# Exercise 1

## TWAP Schedule

In [3]:
twap: {[trade;adv;size]
        time_ct_int : count(trade); / count time units
        traded_size : size%time_ct_int; / size divide to get trading schdule
        time_ct_int#traded_size}

## VWAP Schedule

In [4]:
vwap:{[trade;adv;size]
    vol_weight : abs(trade)%adv; / trade percentage as weight
    vol_weight*size}

## PoV Schedule

In [5]:
part_rate_float : 0.1
pov:{[trade;adv;size]
    deltas (sums part_rate_float*abs trade) & size / &: return samller one; deltas: return difference between adjacent value
    }

## Simulation with 3 schedules

In [6]:
/ use stock with id 0
size: 1000000
tb1_4: select twap_schedule : twap[trade;adv;size],
                    vwap_schedule : vwap[trade;adv;size],
                    pov_schedule : pov[trade;adv;size] from bin10 where id=0


In [7]:
tb1_4

twap_schedule vwap_schedule pov_schedule
----------------------------------------
427.1679      4045.49       45407.18    
427.1679      1530.925      17183.33    
427.1679      2.683672      30.12193    
427.1679      702.8119      7888.465    
427.1679      220.1106      2470.554    
427.1679      821.142       9216.619    
427.1679      203.3384      2282.301    
427.1679      44.01387      494.0182    
427.1679      371.448       4169.188    
427.1679      4809.508      53982.64    
427.1679      480.071       5388.389    
427.1679      2823.343      31689.62    
427.1679      78.11536      876.7784    
427.1679      1179.669      13240.78    
427.1679      463.7469      5205.164    
427.1679      3929.767      44108.28    
427.1679      1.626987      18.26154    
427.1679      37.38737      419.6414    
427.1679      1005.862      11289.95    
427.1679      454.7246      5103.896    
..


## 

In [8]:
tb1_5: select  vwap_schedule : vwap[trade;adv;0.1* adv],
                        pov_schedule : pov[trade;adv;0.1*adv] from bin10 where id=0

In [9]:
tb1_5

vwap_schedule pov_schedule
--------------------------
45407.18      45407.18    
17183.33      17183.33    
30.12193      30.12193    
7888.465      7888.465    
2470.554      2470.554    
9216.619      9216.619    
2282.301      2282.301    
494.0182      494.0182    
4169.188      4169.188    
53982.64      53982.64    
5388.389      5388.389    
31689.62      31689.62    
876.7784      876.7784    
13240.78      13240.78    
5205.164      5205.164    
44108.28      44108.28    
18.26154      18.26154    
419.6414      419.6414    
11289.95      11289.95    
5103.896      5103.896    
..


### The schedules given by the two strategies under perfect volume prediction are the same.

# Exercise 2

## Computation of Impact adjusted price

In [10]:
/ Impact model with given parameter: lambda = 0.8, halflife = 60 mins
computeImpact: {[trade;vol;adv;h] 
    dt:1;
    beta: (log 2)%h*6;
    dv:vol*sqrt (6*60*6.5);
    lamb:0.8;
    b:beta*dt;
    (neg count trade)#ema[b;][0,(lamb*dv*trade%adv)%b]}

In [11]:
tb2: update twap_trade: twap[trade;adv;0.1*adv] by id from bin10 /computing twap trade (historical strategy) schedule 

In [12]:
tb2

time     id date       trade     mid      spread        vol         adv      ..
-----------------------------------------------------------------------------..
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e..
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e..
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e..
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e..
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e..
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e..
09:31:00 0  2019.01.03 -22823.01 91.56276 0.000663703   0.000446367 1.122415e..
09:31:10 0  2019.01.03 -4940.182 91.59088 0.0005615949  0.000446367 1.122415e..
09:31:20 0  2019.01.03 -41691.88 91.59322 0.0005360678  0.000446367 1.122415e..
09:31:30 0  2019.01.03 -539826.4 91.60727 0.0007402842  0.000446367 1.122415e..
09:31:40 0  2019.01.03 -53883.89 91.5487

In [13]:
/ caculate historical strategy's impact
tb2_1: update twap_impact:computeImpact[twap_trade;vol;adv;60] by id from tb2
/mid price - twap impact to get fundamenal price
tb2_1: update Impact_adjusted_price: mid-twap_impact by id from tb2_1

tb2_1

time     id date       trade     mid      spread        vol         adv      ..
-----------------------------------------------------------------------------..
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e..
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e..
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e..
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e..
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e..
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e..
09:31:00 0  2019.01.03 -22823.01 91.56276 0.000663703   0.000446367 1.122415e..
09:31:10 0  2019.01.03 -4940.182 91.59088 0.0005615949  0.000446367 1.122415e..
09:31:20 0  2019.01.03 -41691.88 91.59322 0.0005360678  0.000446367 1.122415e..
09:31:30 0  2019.01.03 -539826.4 91.60727 0.0007402842  0.000446367 1.122415e..
09:31:40 0  2019.01.03 -53883.89 91.5487

## New price with twap schedule impact

In [14]:
/ impacted price constructed by adding fundamental price with impact from twap schedule
tb2_2:update twap_20_trade: twap[trade;adv;0.2*adv] by id from tb2_1
tb2_2:update twap_20_impact: computeImpact[twap_20_trade;vol;adv;60] by id from tb2_2
tb2_2 :update twap_price: twap_20_impact+Impact_adjusted_price from tb2_2

In [15]:
tb2_2

time     id date       trade     mid      spread        vol         adv      ..
-----------------------------------------------------------------------------..
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e..
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e..
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e..
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e..
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e..
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e..
09:31:00 0  2019.01.03 -22823.01 91.56276 0.000663703   0.000446367 1.122415e..
09:31:10 0  2019.01.03 -4940.182 91.59088 0.0005615949  0.000446367 1.122415e..
09:31:20 0  2019.01.03 -41691.88 91.59322 0.0005360678  0.000446367 1.122415e..
09:31:30 0  2019.01.03 -539826.4 91.60727 0.0007402842  0.000446367 1.122415e..
09:31:40 0  2019.01.03 -53883.89 91.5487

## New price with vwap schedule impact

In [16]:
/ impacted price constructed by adding fundamental price with impact from vwap schedule
tb2_3:update vwap_10_trade:vwap[trade;adv;0.1*adv] by id from tb2_2
tb2_3:update vwap_impact: computeImpact[vwap_10_trade;vol;adv;60] by id from tb2_3
tb2_3:update vwap_price: vwap_impact+Impact_adjusted_price from tb2_3

In [17]:
tb2_3

time     id date       trade     mid      spread        vol         adv      ..
-----------------------------------------------------------------------------..
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e..
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e..
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e..
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e..
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e..
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e..
09:31:00 0  2019.01.03 -22823.01 91.56276 0.000663703   0.000446367 1.122415e..
09:31:10 0  2019.01.03 -4940.182 91.59088 0.0005615949  0.000446367 1.122415e..
09:31:20 0  2019.01.03 -41691.88 91.59322 0.0005360678  0.000446367 1.122415e..
09:31:30 0  2019.01.03 -539826.4 91.60727 0.0007402842  0.000446367 1.122415e..
09:31:40 0  2019.01.03 -53883.89 91.5487

## historical strategy's average impact costs

In [18]:
deltas0:{
            0^first[x] -': x} /replace nan , first value, with 0

### Impact cost = $(I_{n-1}+0.5*\Delta_nI)\Delta_nQ=0.5*(I_{n-1}+I_n)\Delta_nQ$

In [19]:
tb2_4: select hist_impact_cost:sum 0.5*((1 xprev twap_impact)+twap_impact)*twap_trade,
                        twap_impact_cost: sum  0.5*((1 xprev twap_20_impact)+twap_20_impact)*twap_20_trade,
                        vwap_impact_cost:sum 0.5*((1 xprev vwap_impact)+vwap_impact)*vwap_10_trade
            by id from tb2_3
tb2_4

id| hist_impact_cost twap_impact_cost vwap_impact_cost
--| --------------------------------------------------
0 | 3358.554         13434.22         3347.984        
1 | 866.2965         3465.186         816.4247        
2 | 448.7952         1795.181         445.82          
3 | 373.1715         1492.686         371.5399        
4 | 495.8013         1983.205         495.4129        
5 | 274.6777         1098.711         271.5745        
6 | 570.735          2282.94          547.8297        
7 | 280.9139         1123.656         273.6034        
8 | 1734.306         6937.222         1727.165        
9 | 832.5131         3330.052         801.4711        
10| 4133.657         16534.63         4031.517        
11| 1504.229         6016.916         1472.007        
12| 768.9616         3075.846         748.9827        
13| 492.3009         1969.203         506.1516        
14| 33873.31         135493.2         32622.74        
15| 1823.752         7295.009         1799.466        
16| 12035.

In [20]:
tb2_4_2: select hist_impact_cost: avg hist_impact_cost,
                        twap_impact_cost: avg twap_impact_cost,
                        vwap_impact_cost: avg vwap_impact_cost
            from tb2_4
tb2_4_2

hist_impact_cost twap_impact_cost vwap_impact_cost
--------------------------------------------------
1260.103         5040.41          1223.044        


## historical strategy's average P&L

### P&L = $Q_{n-1}\Delta_nS-0.5*(I_{n-1}+I_n)\Delta_nQ$

In [21]:
tb2_5: select hist_pnl:sum ((sums 1 xprev twap_trade) * deltas0[Impact_adjusted_price])-0.5*((1 xprev twap_impact)+twap_impact)*twap_trade,
                        twap_pnl: sum ((sums 1 xprev twap_20_trade) * deltas0[Impact_adjusted_price])- 0.5*((1 xprev twap_20_impact)+twap_20_impact)*twap_20_trade,
                        vwap_pnl:sum ((sums 1 xprev vwap_10_trade) * deltas0[Impact_adjusted_price])-0.5*((1 xprev vwap_impact)+vwap_impact)*vwap_10_trade
            by id from tb2_3
tb2_5

id| hist_pnl       twap_pnl       vwap_pnl      
--| --------------------------------------------
0 | -7891535       -1.578979e+007 -9268188      
1 | -1992323       -3986378       -2401699      
2 | -304490.5      -609878.5      -247643.7     
3 | -201442.2      -403630.8      -201335.8     
4 | -72147.09      -145285.8      -111040.9     
5 | -106998.6      -214546.5      -96305.88     
6 | -610545.8      -1222233       -623465       
7 | -54423.89      -109409.6      -53741.18     
8 | -6.499172e+007 -1.299869e+008 -5.097454e+007
9 | -1511281       -3024226       -1413493      
10| -4.825846e+007 -9.652518e+007 -4.355535e+007
11| -5347013       -1.069703e+007 -4866449      
12| -948049.9      -1897638       -774795.9     
13| -1517818       -3036621       -1198468      
14| -8.026742e+007 -1.606026e+008 -7.925849e+007
15| -1177120       -2357887       -1107397      
16| -4408824       -8841719       -3945615      
17| -361639        -724018.3      -333231.9     
18| -317909.6      -

In [22]:
tb2_5:select hist_pnl: avg hist_pnl,twap_pnl:avg twap_pnl,vwap_pnl:avg vwap_pnl from tb2_5

In [23]:
tb2_5

hist_pnl      twap_pnl       vwap_pnl
-------------------------------------
-1.00401e+007 -2.008272e+007 -9047580
